## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-15-12-55-00 +0000,wsj,Consumer prices rose 2.7% in June from a year ...,https://www.wsj.com/economy/inflation-hit-2-7-...
1,2025-07-15-12-49-32 +0000,bbc,UK set up secret Afghan relocation scheme afte...,https://www.bbc.com/news/articles/cvg8zy78787o
2,2025-07-15-12-36-02 +0000,bbc,Ukrainians unimpressed by Trump's 50-day ultim...,https://www.bbc.com/news/articles/c3072ezle7yo
3,2025-07-15-12-35-46 +0000,nypost,Driver killed when tractor-trailer plunged off...,https://nypost.com/2025/07/15/us-news/driver-k...
4,2025-07-15-12-29-45 +0000,bbc,Israel bombs Syrian forces entering Druze city...,https://www.bbc.com/news/articles/c89e3j4e911o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2313/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
19,trump,13
126,tariff,6
10,up,5
9,uk,4
24,killed,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
37,2025-07-15-03-45-00 +0000,wsj,China’s economy showed resilience in a turbule...,https://www.wsj.com/world/china/chinas-economy...,46
70,2025-07-14-14-48-00 +0000,wsj,Trump is threatening to ramp up the U.S. trade...,https://www.wsj.com/economy/trade/european-uni...,44
19,2025-07-15-10-01-45 +0000,bbc,Trump shrugs off giving Parliament speech when...,https://www.bbc.com/news/articles/cx2g1xexe7qo,29
23,2025-07-15-09-38-38 +0000,bbc,'Not our war' - Trump's Nato weapons deal for ...,https://www.bbc.com/news/articles/c14e2ydv4d6o,28
43,2025-07-15-02-01-34 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o,26


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
37,46,2025-07-15-03-45-00 +0000,wsj,China’s economy showed resilience in a turbule...,https://www.wsj.com/world/china/chinas-economy...
70,31,2025-07-14-14-48-00 +0000,wsj,Trump is threatening to ramp up the U.S. trade...,https://www.wsj.com/economy/trade/european-uni...
3,20,2025-07-15-12-35-46 +0000,nypost,Driver killed when tractor-trailer plunged off...,https://nypost.com/2025/07/15/us-news/driver-k...
47,18,2025-07-15-01-25-19 +0000,nypost,Democratic city officers flocking to cop-frien...,https://nypost.com/2025/07/14/us-news/blue-cit...
13,17,2025-07-15-11-24-31 +0000,nypost,Camp Mystic co-owner waited 45 minutes to evac...,https://nypost.com/2025/07/15/us-news/camp-mys...
17,16,2025-07-15-10-51-57 +0000,nypost,Moment Travis Decker’s ‘lookalike’ realizes he...,https://nypost.com/2025/07/15/us-news/moment-t...
31,14,2025-07-15-05-24-07 +0000,nypost,California professor arrested for allegedly to...,https://nypost.com/2025/07/15/us-news/californ...
6,14,2025-07-15-12-27-16 +0000,nypost,NYC saw its 2nd wettest hour in history as rai...,https://nypost.com/2025/07/15/us-news/nyc-saw-...
57,14,2025-07-14-20-35-00 +0000,bbc,Gaza father's outrage after Israeli strike kil...,https://www.bbc.com/news/articles/ckglpk9xjewo
24,13,2025-07-15-09-35-47 +0000,nypost,Former college football star shot and killed i...,https://nypost.com/2025/07/15/us-news/former-t...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
